# Airline Network Revenue Management

In this notebook, I will implement linear programming approach for a simple airline network revenue management. The example is taken from the book [Pricing and Revenue Optimization](https://www.sup.org/books/title/?id=31628) - Second Edition, pages 248-253. This example is in the chapter 10 of the book where the author Robert Phillips discusses network revenue management.

---

## Example 10.1
Consider a simple [hub-and-spoke](https://en.wikipedia.org/wiki/Spoke%E2%80%93hub_distribution_paradigm) airline network where an airline offers two flights:

San Francisco (SFO) ----------> Denver (DEN) ----------------> St. Louis (STL)

*Flight 1*: SFO ---> DEN

*Flight 2*: DEV ---> STL

Given these two flights, the airline can create three products:
- San Francisco to Denver flight
- Denver to St. Louis flight
- San Francisco to St. Louis flight

Essentially an airline product is simply an origin-destination combination. For each product, the airline can offer different fare classes, like full fare, discount fare, and deep discount fare, where in general the price is getting lower and lower, respectively. For this example, we assume that this airline offers full fare and discount fare. Accoridngly, the airline can sell six types of flight tickets to the customers:
1. SFO ---> DEV full fare
2. SFO ---> DEV discount fare
3. DEV ---> STL full fare
4. DEV ---> STL discount fare
5. SFO ---> STL full fare
6. SFO ---> STL discount fare

This combination of product and fare class is called *origin-destination fare class*, or ODF in traditional revenue management. Several ket facts about those ODFs:
- Each ODF is a combination of product and fare class.
- Each ODF has an associated demand. Demands are observed in the ODF level.
- Airline fares are priced at ODF level.
- Each ODF consumes one or multiple resources. For instance, SFO ---> DEV full fare consumes one seat from SFO ---> DEV flight; SFO ---> STL discount fare consumes one flight seat from each of the SFO ---> DEV flight and DEV --> STL flight.

---

The way airline bookings works is that consumers make advance purchase requests on an upcoming flight and airlines make decisions on accepting or rejecting those advance purchases. The tradeoff airlies face is that if they accept a request with discount fare, it might dislace a potential full fare request in the future. Also, if a customer wants to book a fligt that spans multiple legs, it is essentially consuming multiple legs. For airlines the revenue from such booking is lower than the sum of the fare they can collect by selling each resource individually to local booking requests. For instance, airlines might accept a SFo ---> STL booking request for 300.00 dollars where they could have sold two separate single-leg local flights for a total of 350 dollars, 200.00 for the first flight and 150.00 for the second flight. This means lost revenue for the airline.

---

That's why we need a systematic way of evaluating our decision, and that is where linear programming comes in. Before we start, let's provide some basic information about each ODFs of the airline:

| Number | ODF                     | Fare (\$) | Demand |
|--------|-------------------------|-----------|--------|
| 1      |SFO -> DEV full fare     | 150       | 30     |
| 2      |SFO -> DEV discount fare | 100       | 60     |
| 3      |DEV -> STL full fare     | 120       | 20     |
| 4      |DEV -> STL discount fare | 80        | 80     |
| 5      |SFO -> STL full fare     | 250       | 30     |
| 6      |SFO -> STL discount fare | 170       | 40     |

SFO -> DEV flight has a capacity of 100 seats and DEV -> STL flight has a capacity of 120 seats. 

---

Let's define some general notation to this type of problem, then we come back to our problem. 

We first have resources and ODFs. Let's assume we have $m$ resouces (in our case $m=2$) and $n$ ODFs (in our case $n=6$) with $n \geq m$. We can use subscript $i$ to index resources and the subscript $j$ to index ODF. Each resource $i$ has a limited capacity of $C_i>0$. Each ODF has a *known* demand $d_j>0$ and a net contribution margin of $p_j>0$. 

Let $x_j\geq0$ be the amount of ODF $j$ we will allow to book. Then our problem is to find the values of $x_j$ for $j=1, 2, ..., n$ that maximizes total net contriution subjest to the constrained available capacity.

We need one more variable to address the resource consumption instance of each ODF. To do this, we define the incidence variable $a_{ij}$ as follows:
$$
\begin{equation}
a_{ij} = \begin{cases}
1 \qquad & \text{ if resource } i \text{ is used in ODF }j\\
0 \qquad &\text{otherwise}
\end{cases}
\end{equation}
$$

---
Put the incidence variable in the context of our example, it looks like this:

|              |   |   | ODF ($j$)|   |   |   |
|--------------|---|---|----------|---|---|---|
|Resource ($i$)| 1 | 2 | 3        | 4 | 5 | 6 |
|--------------|---|---|----------|---|---|---|
|1             | 1 | 1 | 0        | 0 | 1 | 1 |
|2             | 0 | 0 | 1        | 1 | 1 | 1 |

For instance, both SFO -> DEV full and discount fare consumes a flight seat on SFO -> DEV flight.

Once we have all these, we are now ready to write our linear programming formulation:

$$
\begin{align}
&\max_{x_j}\,\sum_{j=1}^{n}p_jx_j,\\
\text{subject to} &\\
&\sum_{j=1}^{n}a_{ij}x_j \leq C_i\qquad \text{ for all }i,\\
& x_j \leq d_j, \qquad \text{ for all }j,\\
& x_j \geq 0 , \qquad \text{ for all }j.\\
\end{align}
$$

Next, we go back to our example and try to write the airline's problem in a more understandable way. After all, not everyone likes this *complex* notation of LP.

$$
\begin{align}
\text{maximize } \qquad & 150x_1 + 100x_2 + 120x_3 + 80x_4 + 250x_5 + 170x_6\\
\text{subject to } \qquad &\\
& x_1 + x_2 + x_5 + x_6 \leq 100,\\
& x_3 + x_4 + x_5 +x_6 \leq 120,\\
& x_1 \leq 30,\\
& x_2 \leq 60,\\
& x_3 \leq 20,\\
& x_4 \leq 80,\\
& x_5 \leq 30,\\
& x_6 \leq 40,\\
& x_1, x_2, x_3, x_4, x_5, x_6 \geq 0.
\end{align}
$$

Looking back our simple LP example, we can write it as a matrix form so that its python implementation will be very clear. The matrix form is for constraints $Gx\leq h$.

$$
\begin{bmatrix}
1 & 1 & 0 & 0 & 1 & 1\\
0 & 0 & 1 & 1 & 1 & 1\\
1 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4\\
x_5\\
x_6
\end{bmatrix}
\leq
\begin{bmatrix}
100\\
120\\
30\\
60\\
20\\
80\\
30\\
40
\end{bmatrix}
$$

## Python Implementation

In [1]:
# Importing the necessary libraries
import numpy as np
from cvxopt import matrix, solvers

In [2]:
np.set_printoptions(suppress=True)

In [3]:
# Suppress unnecessary output
np.set_printoptions(suppress=True)

# Define the coefficients of the objective function
# Each coefficient represents the fare for an ODF
c = matrix([-150., -100., -120., -80., -250., -170.])

# Define the constraints matrix
# Each row represents a resource constraint or a demand constraint for an ODF
G = matrix([
    [1., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0., 0., 0.], 
    [1., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0., 0.], 
    [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0.],
    [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0.],
    [1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0.],
    [1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1.]
])

# Define the RHS of the constraints
# Each value represents the capacity of a resource or the demand for an ODF
h = matrix([100., 120., 30., 60., 20., 80., 30., 40., 0., 0., 0., 0., 0., 0.])

try:
    # Solve the linear program and print the solution
    sol = solvers.lp(c, G, h)
    print(sol['x'])
except Exception as e:
    print("An error occurred while solving the LP problem.")
    print(f"Error details: {e}")

In [4]:
print(G)

[ 1.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00  1.00e+00]
[ 0.00e+00  0.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00]
[ 1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00]
[-1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00]



In [5]:
sol = solvers.lp(c, G, h)
print(sol['x'])
print(-sol['primal objective'])

     pcost       dcost       gap    pres   dres   k/t
 0: -2.1141e+04 -4.2921e+04  1e+04  0e+00  4e-01  1e+00
 1: -2.3166e+04 -2.6553e+04  2e+03  4e-16  6e-02  2e+01
 2: -2.3784e+04 -2.4468e+04  4e+02  2e-16  1e-02  6e+00
 3: -2.3975e+04 -2.4106e+04  7e+01  3e-16  3e-03  1e+00
 4: -2.3999e+04 -2.4002e+04  1e+00  1e-16  4e-05  5e-02
 5: -2.4000e+04 -2.4000e+04  1e-02  3e-16  4e-07  5e-04
 6: -2.4000e+04 -2.4000e+04  1e-04  1e-16  4e-09  5e-06
Optimal solution found.
[ 3.00e+01]
[ 4.00e+01]
[ 2.00e+01]
[ 7.00e+01]
[ 3.00e+01]
[ 8.85e-07]

23999.999948364362


## The optimal allocation decision is 30 for ODF 1, 40 for ODF 2, 20 for ODF 3, 70 for ODF 4, 30 for ODF 5, and 0 for ODF 6. 
## With this allocation the airline can achieve a revenue of \$24,000.


The output of the LP solver is a list of quantities for each of the six origin-destination fare classes (ODFs) that maximizes the airline's revenue. Each quantity represents the number of seats the airline should sell for that ODF. The primal objective value is the maximum revenue that the airline can achieve with this allocation of seats.


In [6]:
print(sol['z']) # DUAL VALUES

[ 1.00e+02]
[ 8.00e+01]
[ 5.00e+01]
[ 5.16e-07]
[ 4.00e+01]
[ 1.70e-06]
[ 7.00e+01]
[ 1.76e-07]
[ 1.87e-07]
[ 1.18e-07]
[ 3.23e-07]
[ 1.91e-07]
[ 1.65e-07]
[ 1.00e+01]



## Example 10.2

In [7]:
# Define the coefficients of the objective function for the second example
c_max = matrix([180., 160., 140., 130., 100., 130., 110., 90., 80., 75., 260., 240., 200., 190., 170.])
c = - c_max

# Define the constraints matrix for the second example
demand_identity = np.identity(15)  # demand constraint coefficients
non_negative_identity = -np.identity(15)  # non negativity constraint coefficients
cap_constraint = matrix([  # resource constraint coefficients
    [1, 0],
    [1, 0],
    [1, 0],
    [1, 0],
    [1, 0],
    [0, 1],
    [0, 1],
    [0, 1],
    [0, 1],
    [0, 1],
    [1, 1],
    [1, 1],
    [1, 1],
    [1, 1],
    [1, 1]
])
# Combine three matrices to get the final coefficient matrix for the constraints LHS
G = np.concatenate((cap_constraint, demand_identity, non_negative_identity), axis=0)
G = matrix(G)

# Define the RHS of the constraints for the second example
cap = np.array([100., 120.])
demand1 = np.array([10., 20., 15., 18., 27.])
demand2 = np.array([15., 20., 15., 20., 30.])
demand3 = np.array([20., 10., 10., 15., 15.])
zeros = np.zeros(15)
h = np.concatenate((cap, demand1, demand2, demand3, zeros), axis=0)
h = matrix(h)

try:
    # Solve the linear program and print the solution for the second example
    sol = solvers.lp(c, G, h)
    print(sol['x'])  # optimal solution
    print(sol['z'])  # DUAL VALUES
    print(-sol['primal objective'])  # maximum revenue
except Exception as e:
    print("An error occurred while solving the LP problem.")
    print(f"Error details: {e}")

In [8]:
sol = solvers.lp(c, G, h)
print(sol['x']) # optimal solution
print(sol['z']) # DUAL VALUES
print(-sol['primal objective']) # maximum revenue

     pcost       dcost       gap    pres   dres   k/t
 0: -2.3366e+04 -3.9789e+04  9e+03  0e+00  3e-01  1e+00
 1: -2.5702e+04 -2.9810e+04  2e+03  1e-16  8e-02  4e+01
 2: -2.6353e+04 -2.6973e+04  3e+02  8e-17  1e-02  6e+00
 3: -2.6499e+04 -2.6565e+04  4e+01  2e-16  1e-03  7e-01
 4: -2.6514e+04 -2.6518e+04  2e+00  1e-16  7e-05  4e-02
 5: -2.6515e+04 -2.6515e+04  2e-02  1e-16  7e-07  4e-04
 6: -2.6515e+04 -2.6515e+04  2e-04  3e-16  7e-09  4e-06
Optimal solution found.
[ 1.00e+01]
[ 2.00e+01]
[ 1.50e+01]
[ 1.80e+01]
[ 2.22e-07]
[ 1.50e+01]
[ 2.00e+01]
[ 1.50e+01]
[ 2.00e+01]
[ 1.30e+01]
[ 2.00e+01]
[ 1.00e+01]
[ 7.00e+00]
[ 3.70e-07]
[ 1.86e-07]

[ 1.25e+02]
[ 7.50e+01]
[ 5.50e+01]
[ 3.50e+01]
[ 1.50e+01]
[ 5.00e+00]
[ 3.20e-07]
[ 5.50e+01]
[ 3.50e+01]
[ 1.50e+01]
[ 5.00e+00]
[ 4.84e-07]
[ 6.00e+01]
[ 4.00e+01]
[ 5.54e-06]
[ 2.14e-07]
[ 6.21e-07]
[ 5.85e-07]
[ 2.69e-07]
[ 3.51e-07]
[ 2.57e-07]
[ 2.50e+01]
[ 4.05e-07]
[ 2.66e-07]
[ 3.68e-07]
[ 2.24e-07]
[ 6.77e-07]
[ 2.70e-07]
[ 5.28e-07]
[


Again, the output of the LP solver is a list of quantities for each of the 15 ODFs that maximizes the airline's revenue. Each quantity represents the number of seats the airline should sell for that ODF. The primal objective value is the maximum revenue that the airline can achieve with this allocation of seats. The dual values can be used to assess the sensitivity of the optimal solution to changes in the constraints.



## Output Interpretation

The output of the linear programming solution consists of two arrays and a scalar value. 

The first array represents the optimal solution, i.e., the number of tickets to be sold for each origin-destination fare class (ODF). For example, the first element of the array corresponds to the number of tickets for the first ODF, the second element corresponds to the second ODF, and so on.

The second array represents the dual values, or shadow prices, of the constraints. These values give us information about how much the objective function would change if we were to change the right-hand side of the constraints by one unit. They can be interpreted as the marginal values of the resources.

The scalar value represents the maximum revenue that can be achieved with the optimal ticket allocation.

These results can be used by the airline to make informed decisions about ticket allocation in order to maximize revenue.
